# TODO:  Deploy Multi-Model Endpoint Using 2 Trained TensorFlow Models

https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst#deploying-more-than-one-model-to-your-endpoint

In [ ]:
!pip install -q --upgrade pip
!pip install -q wrapt --upgrade --ignore-installed
!pip install -q tensorflow==2.1.0
!pip install -q transformers==2.8.0

In [ ]:
!pip install --upgrade sagemaker==1.56.1

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Configure First Training Job

In [ ]:
%store -r training_job_name

In [ ]:
print(training_job_name)

In [ ]:
model1_s3_uri = 's3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name)

In [ ]:
# TODO:  Add a second training job

In [ ]:
%store -r training_job_name2

In [ ]:
print(training_job_name2)

In [ ]:
model2_s3_uri = 's3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name2)

# TODO:  Access the model.tar.gz from each of the S3 locations derived from the 2 training job names

Put them in ./multi/model1 and model2/ directory structure shown here

└── multi

    ├── model1
  
    │   └── <version number>
    
    │       ├── saved_model.pb
  
    │       └── variables
  
    │           └── ...
  
    └── model2
  
        └── <version number>
      
            ├── saved_model.pb
          
            └── variables
          
                └── ...
              

In [ ]:
%bash

aws s3 cp $model1_s3_uri model1.tar.gz
aws s3 cp $model2_s3_uri model2.tar.gz

mkdir -p multi/model1
mkdir -p multi/model2

tar xvf model1.tar.gz -C ./multi/model1
tar xvf model2.tar.gz -C ./multi/model2


In [ ]:
# TODO:  Change `export/Servo` to whatever matches our models up to the 0/saved_model dir, I believe.
# mv multi/model1/export/Servo/* multi/model1/
# mv multi/model2/export/Servo/* multi/model2/

# rm -rf multi/model1/export
# rm -rf multi/model2/export

# Repackage the models into a new multi.tar.gz 

In [ ]:
# tar -C "./multi/" -czvf multi.tar.gz multi/

# Upload the New Archive to S3

In [ ]:
multi_model_s3_uri = #

In [ ]:
!aws s3 cp multi.tar.gz $multi_model_s3_uri


# Create SageMaker Model from the Multi-Model

In [ ]:
from sagemaker.tensorflow.serving import Model, Predictor

multi_model_s3_uri = # 's3://mybucket/models/multi.tar.gz'

# For multi-model endpoints, you should set the default model name in
# an environment variable. If it isn't set, the endpoint will work,
# but the model it will select as default is unpredictable.
env = {
  'SAGEMAKER_TFS_DEFAULT_MODEL_NAME': 'model1'  # <== This must match the directory
}

model = Model(model_data=model_data, 
              role=role, 
              framework_version='2.1.0', 
              env=env)


# Deploy the Multi-Model as a SageMaker Endpoint

In [ ]:
predictor = model.deploy(instance_type='ml.c5.xlarge',
                         initial_instance_count=2)
